In [9]:
import requests
import json
import pandas as pd
import ssl

try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize

In [70]:

def krx_stock_listing():
    # KRX 상장회사목록
    # For mac, SSL CERTIFICATION VERIFICATION ERROR
    ssl._create_default_https_context = ssl._create_unverified_context

    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
    df_listing = pd.read_html(url, header=0)[0]
    cols_ren = {'회사명':'Name', '종목코드':'Symbol', '업종':'Sector', '주요제품':'Industry', 
                        '상장일':'ListingDate', '결산월':'SettleMonth',  '대표자명':'Representative', 
                        '홈페이지':'HomePage', '지역':'Region', }
    df_listing = df_listing.rename(columns = cols_ren)
    df_listing['Symbol'] = df_listing['Symbol'].apply(lambda x: '{:06d}'.format(x))
    df_listing['ListingDate'] = pd.to_datetime(df_listing['ListingDate'])

    # KRX 주식종목검색
    data = {'bld': 'dbms/comm/finder/finder_stkisu',}
    r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', data=data)

    jo = json.loads(r.text)
    df_finder = json_normalize(jo, 'block1')

    # full_code, short_code, codeName, marketCode, marketName, marketEngName, ord1, ord2
    df_finder.columns = ['FullCode', 'Symbol', 'Name', 'MarketCode', 'MarketName', 'Market', 'Ord1', 'Ord2']

    # 상장회사목록, 주식종목검색 병합
    df_left = df_finder[['Symbol', 'Market', 'Name']]
    df_right = df_listing[['Symbol', 'Sector', 'Industry', 'ListingDate', 'SettleMonth', 'Representative', 'HomePage', 'Region']]

    df_master = pd.merge(df_left, df_right, how='left', left_on='Symbol', right_on='Symbol')
    
    col_map={'Symbol':'종목코드', 
             'Market':'시장구분', 
             'Name':'종목명', 
             'Sector':'섹터', 
             'Industry':'산업', 
             'ListingDate':'상장일',
             'SettleMonth':'결산일', 
             'Representative':'대표', 
             'HomePage':'HomePage', 
             'Region':'지역'}
    
    df_master = df_master.rename(columns=col_map)
    # if self.market in ['KONEX', 'KOSDAQ', 'KOSPI']:
    #     return df_master[df_master['Market'] == self.market] 
    return df_master

In [11]:
# https://progr-account.tistory.com/151
def get_stock_Indices(date):  ### date type : str(),,, ex) '2010722'

    url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

    data = {'bld': 'dbms/MDC/STAT/standard/MDCSTAT00101',
            'idxIndMidclssCd': '01',
            'trdDd': date,  ### date type : str(),,, ex) '2010722'
            'share': '2',
            'money': '3',
            'csvxls_isNo': 'false'}

    response = requests.post(url, data=data)  ### get이 아님에 유의
    stock_data = response.json()['output']  ### 불러온 정보를 json으로 추출하면 dict()구조인데 원하는 정보는 key:'output'에 있다.

    return stock_data


def get_stock_sise(date):  ### date type : str(),,, ex) '2010722'

    url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

    data = {'bld': 'dbms/MDC/STAT/standard/MDCSTAT01501',
            'locale': 'ko_KR',
            'mktId': 'ALL',
            'trdDd': date,  ### date type : str(),,, ex) '2010722'
            'share': '1',
            'money': '1',
            'csvxls_isNo': 'false'}

    response = requests.post(url, data=data)  ### get이 아님에 유의
    stock_data = response.json()  ### 불러온 정보를 json으로 추출하면 dict()구조인데 원하는 정보는 key:'output'에 있다.
    
    col_map={'ISU_SRT_CD': '종목코드',
        'ISU_ABBRV': '종목명',
        'MKT_NM': '시장구분',
        'SECT_TP_NM': '소속부',
        'TDD_CLSPRC': '종가',
        'FLUC_TP_CD': 'FLUC_TP_CD',
        'CMPPREVDD_PRC': '대비',
        'FLUC_RT': '등락률',
        'TDD_OPNPRC': '시가',
        'TDD_HGPRC': '고가',
        'TDD_LWPRC': '저가',
        'ACC_TRDVOL': '누적거래량',
        'ACC_TRDVAL': '누적거래대금',
        'MKTCAP': '시가총액',
        'LIST_SHRS': '상장주식수',
        'MKT_ID': '시장ID'}
    
    df = pd.DataFrame(stock_data['OutBlock_1'])
    df = df.rename(columns=col_map)
    df.drop(['소속부', 'FLUC_TP_CD', '시장ID'], axis=1, inplace=True)

    return df


# PER/PBR/배당수익률
# https://blog.naver.com/PostView.naver?blogId=sisomimoctrl&logNo=222414542520&parentCategoryNo=&categoryNo=185&viewDate=&isShowPopularPosts=false&from=postList
def get_stock_Investment_Indicators(date):  ### date type : str(),,, ex) '2010722'

    url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

    data = {
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT03501',
        'searchType': '1',
        'mktId': 'ALL',
        'trdDd': date,  ### date type : str(),,, ex) '2010722'
        'share': '1',
        'csvxls_isNo': 'false'}

    
    response = requests.post(url, data=data)  ### get이 아님에 유의
    stock_data = response.json()  ### 불러온 정보를 json으로 추출하면 dict()구조인데 원하는 정보는 key:'output'에 있다.

    col_map={'ISU_SRT_CD': '종목코드',
        'ISU_ABBRV': '종목명',
        'ISU_ABBRV_STR': 'ISU_ABBRV_STR',
        'TDD_CLSPRC': '종가',
        'FLUC_TP_CD': 'FLUC_TP_CD',
        'CMPPREVDD_PRC': '대비',
        'FLUC_RT': '등락률',
        'EPS': 'EPS',
        'PER': 'PER',
        'FWD_EPS': 'FWD_EPS',
        'FWD_PER': 'FWD_PER',
        'BPS': 'BPS',
        'PBR': 'PBR',
        'DPS': '주당배당금',
        'DVD_YLD': '배당수익률'}
    
    df = pd.DataFrame(stock_data['output'])
    df = df.rename(columns=col_map)
    df.drop(['ISU_ABBRV_STR', 'FLUC_TP_CD'], axis=1, inplace=True)

    return df

In [47]:
df0=get_stock_Investment_Indicators('20220504')
df0

,종목코드,종목명,종가,대비,등락률,EPS,PER,FWD_EPS,FWD_PER,BPS,PBR,주당배당금,배당수익률
0,060310,3S,"3,220",-30,-0.92,16,201.25,-,-,792,4.07,0,0.00
1,095570,AJ네트웍스,"6,500",-70,-1.07,"1,707",3.81,-,-,"8,075",0.80,270,4.15
2,006840,AK홀딩스,"20,850",-400,-1.88,-,-,-,-,"45,961",0.45,200,0.96
3,054620,APS홀딩스,"12,850",-400,-3.02,"1,179",10.90,-,-,"10,088",1.27,100,0.78
4,265520,AP시스템,"23,750",-200,-0.84,"3,932",6.04,"3,723",6.38,"12,713",1.87,240,1.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,000547,흥국화재2우B,"30,500",-350,-1.13,-,-,-,-,-,-,0,0.00
2446,000545,흥국화재우,"8,730",130,1.51,-,-,-,-,-,-,0,0.00
2447,003280,흥아해운,"2,875",-30,-1.03,118,24.36,-,-,428,6.72,0,0.00
2448,037440,희림,"8,410",-170,-1.98,490,17.16,-,-,"4,769",1.76,150,1.78


In [13]:
get_stock_Investment_Indicators('20210504')

,종목코드,종목명,종가,대비,등락률,EPS,PER,FWD_EPS,FWD_PER,BPS,PBR,주당배당금,배당수익률
0,060310,3S,"2,765",-30,-1.07,-,-,-,-,745,3.71,0,0.00
1,095570,AJ네트웍스,"5,820",60,1.04,-,-,-,-,"6,089",0.96,210,3.61
2,006840,AK홀딩스,"30,000",-250,-0.83,-,-,-,-,"50,471",0.59,400,1.33
3,054620,APS홀딩스,"13,700",850,6.61,-,-,-,-,"8,135",1.68,0,0.00
4,265520,AP시스템,"28,050",200,0.72,"1,736",16.16,-,-,"9,041",3.10,120,0.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,000547,흥국화재2우B,"41,500","4,500",12.16,-,-,-,-,-,-,0,0.00
2372,000545,흥국화재우,"12,550",450,3.72,-,-,-,-,-,-,0,0.00
2373,003280,흥아해운,258,0,0.00,-,-,-,-,-,-,0,0.00
2374,037440,희림,"8,670",470,5.73,389,22.29,-,-,"4,034",2.15,150,1.73


In [14]:
df1=get_stock_sise('20210503')
df2=get_stock_sise('20220401')

In [15]:
df1

,종목코드,종목명,시장구분,종가,대비,등락률,시가,고가,저가,누적거래량,누적거래대금,시가총액,상장주식수
0,060310,3S,KOSDAQ,"2,795",-40,-1.41,"2,810","2,810","2,720","254,634","702,615,705","127,665,929,300","45,676,540"
1,095570,AJ네트웍스,KOSPI,"5,760",200,3.60,"5,950","5,980","5,600","381,642","2,195,794,180","269,696,419,200","46,822,295"
2,006840,AK홀딩스,KOSPI,"30,250",-850,-2.73,"31,450","31,550","30,100","109,871","3,379,968,750","400,738,720,250","13,247,561"
3,054620,APS홀딩스,KOSDAQ,"12,850",800,6.64,"12,100","13,050","12,100","332,847","4,263,184,800","262,065,739,850","20,394,221"
4,265520,AP시스템,KOSDAQ,"27,850",-450,-1.59,"28,000","28,700","27,700","203,215","5,718,480,500","403,274,321,950","14,480,227"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,000547,흥국화재2우B,KOSPI,"37,000","4,250",12.98,"31,000","40,000","31,000","63,452","2,320,849,700","5,683,200,000","153,600"
2555,000545,흥국화재우,KOSPI,"12,100",900,8.04,"10,700","12,600","10,700","137,206","1,625,041,100","9,292,800,000","768,000"
2556,003280,흥아해운,KOSPI,258,0,0.00,0,0,0,0,0,"30,117,000,222","116,732,559"
2557,037440,희림,KOSDAQ,"8,200",-50,-0.61,"8,260","8,440","8,040","541,345","4,444,856,330","114,164,295,000","13,922,475"


In [ ]:
key=list(df1['종목코드'])
val=list(df1['종가'])
d1=dict(zip(key,val))

In [ ]:
key=list(df2['종목코드'])
val=list(df2['종가'])
d2=dict(zip(key,val))

In [71]:
df_krx=krx_stock_listing()
df_krx.to_csv('df_krx.csv',encoding='utf-8-sig', index=False)

In [72]:
df_krx

,종목코드,시장구분,종목명,섹터,산업,상장일,결산일,대표,HomePage,지역
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
8054,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8055,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
8056,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시
8057,419890,KOSPI,히어로즈 단기채권ESG액티브,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [28]:
df_krx=pd.read_csv('df_krx.csv',encoding='utf-8-sig')

In [105]:
df_curr=get_stock_sise('20220504')
df0=get_stock_Investment_Indicators('20220504')
# ['EPS', 'PER', 'FWD_EPS', 'FWD_PER', 'BPS', 'PBR', '주당배당금', '배당수익률']
df_master = pd.merge(df_curr, df0[['종목코드', 'EPS', 'PER', 'FWD_EPS', 'FWD_PER', 'BPS', 'PBR', '주당배당금', '배당수익률']], how='left', left_on='종목코드', right_on='종목코드')
df_master = pd.merge(df_master, df_krx[['종목코드', '섹터', '산업']], how='left', left_on='종목코드', right_on='종목코드')
df_master['시가총액'] = df_master['시가총액'].str.replace(',', '').astype(float)
cols = list(df_master.columns)
cols = cols[:3]+cols[-2:]+cols[3:-2]
df_master=df_master[cols]
# # KOSPI KOSDAQ KONEX
df_master=df_master.sort_values(by=['시가총액'], ascending=[False])[df_master['시장구분'] == 'KOSDAQ']

C:\Users\hs-I7\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


In [106]:
df_master.reset_index(drop=True)

,종목코드,종목명,시장구분,섹터,산업,종가,대비,등락률,시가,고가,...,시가총액,상장주식수,EPS,PER,FWD_EPS,FWD_PER,BPS,PBR,주당배당금,배당수익률
0,247540,에코프로비엠,KOSDAQ,일차전지 및 축전지 제조업,양극활물질,"520,100","9,100",1.78,"522,000","532,100",...,1.192041e+13,"22,919,466","4,666",111.47,"8,870",58.64,"24,156",21.53,920,0.18
1,091990,셀트리온헬스케어,KOSDAQ,기타 전문 도매업,바이오의약품 마케팅 및 판매,"60,900","-1,200",-1.93,"62,100","62,600",...,9.629895e+12,"158,126,350",977,62.33,"1,425",42.74,"13,473",4.52,260,0.43
2,066970,엘앤에프,KOSDAQ,일차전지 및 축전지 제조업,2차전지 양극활물질,"233,500","5,100",2.23,"230,300","238,000",...,8.389565e+12,"35,929,616",-,-,"4,137",56.44,"19,899",11.73,0,0.00
3,293490,카카오게임즈,KOSDAQ,소프트웨어 개발 및 공급업,"모바일, PC게임 퍼블리싱","58,500",-600,-1.02,"59,200","60,300",...,4.598502e+12,"78,606,876","7,073",8.27,"3,636",16.09,"27,171",2.15,0,0.00
4,263750,펄어비스,KOSDAQ,소프트웨어 개발 및 공급업,온라인게임 개발 서비스(검은사막),"65,500",-200,-0.30,"66,000","67,300",...,4.337508e+12,"66,221,500",973,67.32,"4,420",14.82,"11,757",5.57,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,347140,케이프이에스제4호,KOSDAQ,기타 금융업,기타금융지원서비스업,"2,175",0,0.00,"2,180","2,180",...,6.544575e+09,"3,009,000",-,-,-,-,-,-,0,0.00
1554,405350,IBKS제17호스팩,KOSDAQ,금융 지원 서비스업,금융 지원 서비스,"2,355",0,0.00,"2,345","2,355",...,6.405600e+09,"2,720,000",-,-,-,-,-,-,0,0.00
1555,021045,대호특수강우,KOSDAQ,NaN,NaN,"1,490",-10,-0.67,"1,495","1,500",...,6.321274e+09,"4,242,466",-,-,-,-,-,-,0,0.00
1556,032685,소프트센우,KOSDAQ,NaN,NaN,"32,100",50,0.16,"32,100","32,450",...,4.585710e+09,"142,857",-,-,-,-,-,-,0,0.00


In [104]:
df_master

,index,종목코드,종목명,시장구분,섹터,산업,종가,대비,등락률,시가,...,시가총액,상장주식수,EPS,PER,FWD_EPS,FWD_PER,BPS,PBR,주당배당금,배당수익률
0,1483,247540,에코프로비엠,KOSDAQ,일차전지 및 축전지 제조업,양극활물질,"520,100","9,100",1.78,"522,000",...,1.192041e+13,"22,919,466","4,666",111.47,"8,870",58.64,"24,156",21.53,920,0.18
1,1136,091990,셀트리온헬스케어,KOSDAQ,기타 전문 도매업,바이오의약품 마케팅 및 판매,"60,900","-1,200",-1.93,"62,100",...,9.629895e+12,"158,126,350",977,62.33,"1,425",42.74,"13,473",4.52,260,0.43
2,1533,066970,엘앤에프,KOSDAQ,일차전지 및 축전지 제조업,2차전지 양극활물질,"233,500","5,100",2.23,"230,300",...,8.389565e+12,"35,929,616",-,-,"4,137",56.44,"19,899",11.73,0,0.00
3,2022,293490,카카오게임즈,KOSDAQ,소프트웨어 개발 및 공급업,"모바일, PC게임 퍼블리싱","58,500",-600,-1.02,"59,200",...,4.598502e+12,"78,606,876","7,073",8.27,"3,636",16.09,"27,171",2.15,0,0.00
4,2278,263750,펄어비스,KOSDAQ,소프트웨어 개발 및 공급업,온라인게임 개발 서비스(검은사막),"65,500",-200,-0.30,"66,000",...,4.337508e+12,"66,221,500",973,67.32,"4,420",14.82,"11,757",5.57,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,2062,347140,케이프이에스제4호,KOSDAQ,기타 금융업,기타금융지원서비스업,"2,175",0,0.00,"2,180",...,6.544575e+09,"3,009,000",-,-,-,-,-,-,0,0.00
1554,92,405350,IBKS제17호스팩,KOSDAQ,금융 지원 서비스업,금융 지원 서비스,"2,355",0,0.00,"2,345",...,6.405600e+09,"2,720,000",-,-,-,-,-,-,0,0.00
1555,555,021045,대호특수강우,KOSDAQ,NaN,NaN,"1,490",-10,-0.67,"1,495",...,6.321274e+09,"4,242,466",-,-,-,-,-,-,0,0.00
1556,1142,032685,소프트센우,KOSDAQ,NaN,NaN,"32,100",50,0.16,"32,100",...,4.585710e+09,"142,857",-,-,-,-,-,-,0,0.00
